In [2]:
!pip install -q ultralytics pyyaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.3 MB/s eta 0:00:00


In [3]:
import torch
import time
import torch.nn as nn
from torchvision import transforms
from torchvision.models import mobilenet_v2
from PIL import Image
from PIL import Image, ImageDraw, ImageFont
import os
from google.colab import drive
from ultralytics import YOLO
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Monta tu Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# --- ¡IMPORTANTE! MODIFICA ESTAS DOS LÍNEAS ---

# 1. Ruta al archivo .pt de tu DETECTOR de heces (entrenado con YOLO)
DETECTOR_MODEL_PATH = "/content/drive/MyDrive/Caso de Estudio - Examen Complexivo/1-resultados_detector_yolov8nano/weights/best.pt"

# Ruta al archivo de tu mejor modelo guardado (.pt)
CLASSIFIER_MODEL_PATH = "/content/drive/MyDrive/Caso de Estudio - Examen Complexivo/2-resultados_clasificacion/MobileNetV2/best_model.pt"

# Ruta a la carpeta que contiene tus nuevas imágenes sin clasificar
IMAGE_DIR_TO_TEST = "/content/drive/MyDrive/Caso de Estudio - Examen Complexivo/Imagenes propias"

# 4. Carpeta en Drive donde se guardarán las imágenes con los resultados
# Se dibujarán los cuadros y las etiquetas sobre las imágenes originales
OUTPUT_DIR = "/content/drive/MyDrive/Caso de Estudio - Examen Complexivo/Resultados_del_sistema_Pipeline_Final"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Asegúrate de que el orden de las clases del CLASIFICADOR sea el mismo que en el entrenamiento
CLASSIFIER_CLASS_NAMES = ['Coccidiosis', 'Newcastle Disease', 'Salmonella', 'Sanas']


In [6]:
# Configura el dispositivo (funcionará en CPU si no hay GPU disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")


Usando dispositivo: cpu


In [7]:
# PASO 2: CARGAR AMBOS MODELOS
# ===================================================================
# --- Cargar Modelo 1: El Detector ---
print("Cargando modelo detector (YOLOv8n)...")
detector_model = YOLO(DETECTOR_MODEL_PATH)
#aplicar el umbral de confianza óptimo que nos dios el entranamiento
# ---results = detection_model(img_path, conf=0.352)
print("Detector cargado.")

# --- Cargar Modelo 2: El Clasificador ---
print(f"Cargando modelo clasificador (MobileNetV2)...")
classifier_model = mobilenet_v2(weights=None)
classifier_model.classifier[1] = nn.Linear(classifier_model.classifier[1].in_features, len(CLASSIFIER_CLASS_NAMES))
classifier_model.load_state_dict(torch.load(CLASSIFIER_MODEL_PATH, map_location=device))
classifier_model.to(device)
classifier_model.eval() # Poner en modo de evaluación
print("Clasificador cargado y en modo de evaluación.")

Cargando modelo detector (YOLOv8n)...
Detector cargado.
Cargando modelo clasificador (MobileNetV2)...
Clasificador cargado y en modo de evaluación.


In [8]:
# PASO 3: DEFINIR TRANSFORMACIONES Y FUNCIÓN DE CLASIFICACIÓN
# ===================================================================
# Transformaciones para las imágenes RECORTADAS que irán al clasificador
classifier_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def classify_crop(image_crop, model, transforms, class_names):
    """Función para clasificar una sola imagen recortada."""
    # La imagen ya viene como un objeto de PIL
    image_tensor = transforms(image_crop).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.softmax(outputs, dim=1)[0]
        confidence, predicted_idx = torch.max(probabilities, 0)
    prediction = class_names[predicted_idx.item()]
    return prediction, confidence.item()


In [10]:
# ===================================================================
# PASO 4: EJECUTAR EL PIPELINE (CON VISUALIZACIÓN MEJORADA)
# ===================================================================
print("\n--- INICIANDO PIPELINE DE ANÁLISIS ---")

# Diccionario para guardar los resultados para el resumen
summary = {
    'Sanas': [], 'Coccidiosis': [], 'Salmonella': [], 'Newcastle Disease': []
}
# Lista para guardar los tiempos de procesamiento para el promedio de FPS
processing_times = []

valid_extensions = ('.jpg', '.jpeg', '.png')
images_shown = 0
max_images_to_show = 5

if not os.path.isdir(IMAGE_DIR_TO_TEST):
    print(f"Error: El directorio '{IMAGE_DIR_TO_TEST}' no existe.")
else:
    for filename in sorted(os.listdir(IMAGE_DIR_TO_TEST)):
        if filename.lower().endswith(valid_extensions):
            # MEJORA: Iniciar el cronómetro para esta imagen
            start_time = time.time()
            full_path = os.path.join(IMAGE_DIR_TO_TEST, filename)
            print(f"\n--- Analizando imagen: {filename} ---")
            original_image = Image.open(full_path).convert('RGB')
            # Prepara la imagen para poder dibujar sobre ella
            draw = ImageDraw.Draw(original_image)
            try:
                # MEJORA: Se aumenta el tamaño de la fuente
                font = ImageFont.truetype("LiberationSans-Regular.ttf", 55)
            except IOError:
                font = ImageFont.load_default()

            # --- Paso 1: Detección de Heces con YOLO ---
            detections = detector_model(original_image, verbose=False)
            if len(detections[0].boxes) == 0:
                print("  >> No se encontraron heces en esta imagen.")
                # Muestra la imagen si está dentro del límite de visualización
                if images_shown < max_images_to_show:
                    plt.figure(figsize=(8, 8))
                    plt.imshow(original_image)
                    plt.title(f"{filename}\n(No se encontraron heces)")
                    plt.axis('off')
                    plt.show()
                    images_shown += 1
                # No se guarda la imagen en Drive y se pasa a la siguiente
            else:
                # --- Paso 2: Iterar, Recortar, Clasificar y Dibujar cada detección ---
                for i, box in enumerate(detections[0].boxes):
                    # Obtener coordenadas
                    coords = box.xyxy[0].tolist()
                    x1, y1, x2, y2 = map(int, coords)
                    # Recortar la región de interés
                    crop = original_image.crop((x1, y1, x2, y2))
                    # Clasificar el recorte para obtener el diagnóstico
                    disease, confidence = classify_crop(crop, classifier_model, classifier_transforms, CLASSIFIER_CLASS_NAMES)
                    # --- MEJORA VISUAL ---
                    label_text = f"{disease} ({confidence*100:.1f}%)"
                    # Dibuja el cuadro delimitador más grueso
                    draw.rectangle([x1, y1, x2, y2], outline="lime", width=10)
                    # Dibuja el texto del diagnóstico con fondo y letra más grande
                    text_bbox = draw.textbbox((x1, y1 - 32), label_text, font=font)
                    draw.rectangle(text_bbox, fill="black")
                    draw.text((x1, y1 - 32), label_text, fill="lime", font=font)
                    # ---------------------

                    # Informe detallado en la consola
                    print(f"  - Detección #{i+1}:")
                    print(f"    >> Diagnóstico: {disease} (Confianza: {confidence*100:.2f}%)")

                    # Guardar el resultado para el resumen
                    summary[disease].append(f"Imagen: {filename} (Detección #{i+1})")

                # Muestra la imagen final con los resultados en la salida de Colab
                if images_shown < max_images_to_show:
                    plt.figure(figsize=(10, 10))
                    plt.imshow(original_image)
                    plt.title(f"Resultado de: {filename}", fontsize=16)
                    plt.axis('off')
                    plt.show()
                    images_shown += 1

                # Guardar la imagen con resultados en tu carpeta de Drive
                output_path = os.path.join(OUTPUT_DIR, f"resultado_{filename}")
                original_image.save(output_path)
                print(f"  >> Imagen con resultados guardada en: {output_path}")

            # MEJORA: Detener el cronómetro y calcular FPS
            end_time = time.time()
            total_time = end_time - start_time
            processing_times.append(total_time)
            fps = 1 / total_time if total_time > 0 else 0

            print(f"  >> Tiempo de procesamiento: {total_time:.2f} segundos (FPS: {fps:.2f})")

print("\n--- ANÁLISIS FINALIZADO ---")

Output hidden; open in https://colab.research.google.com to view.

In [11]:
# ===================================================================
# PASO 5: MOSTRAR RESUMEN FINAL (CON FPS PROMEDIO)
# ===================================================================
print("\n\n" + "="*80)
print("                           RESUMEN FINAL DEL ANÁLISIS")
print("="*80)

# Calcular totales
total_sanas = len(summary['Sanas'])
total_coccidiosis = len(summary['Coccidiosis'])
total_salmonella = len(summary['Salmonella'])
total_newcastle = len(summary['Newcastle Disease'])
total_heces = total_sanas + total_coccidiosis + total_salmonella + total_newcastle
if total_heces > 0:
    print(f"\nSe detectaron un total de {total_heces} heces en las imágenes analizadas.\n")
    summary_data = [
        {"Categoría": "Sanas", "Total de Detecciones": total_sanas, "Imágenes y Detecciones": ", ".join(summary['Sanas'])},
        {"Categoría": "Coccidiosis", "Total de Detecciones": total_coccidiosis, "Imágenes y Detecciones": ", ".join(summary['Coccidiosis'])},
        {"Categoría": "Salmonella", "Total de Detecciones": total_salmonella, "Imágenes y Detecciones": ", ".join(summary['Salmonella'])},
        {"Categoría": "Newcastle Disease", "Total de Detecciones": total_newcastle, "Imágenes y Detecciones": ", ".join(summary['Newcastle Disease'])}
    ]
    # Convertir a un DataFrame de pandas
    df_summary = pd.DataFrame(summary_data)
    # Configurar opciones de visualización para que se muestre todo el texto
    pd.set_option('display.max_colwidth', None)
    # Mostrar la tabla
    display(df_summary)
else:
    print("\nNo se detectó ninguna hez en las imágenes proporcionadas.")
print("\n" + "="*80)

if processing_times:
    avg_time_per_image = sum(processing_times) / len(processing_times)
    avg_fps = 1 / avg_time_per_image if avg_time_per_image > 0 else 0

    print("\n" + "-"*40)
    print("        MÉTRICAS DE VELOCIDAD PROMEDIO")
    print("-" * 40)
    print(f"  Imágenes analizadas: {len(processing_times)}")
    print(f"  Tiempo promedio por imagen: {avg_time_per_image:.3f} segundos")
    print(f"  Fotogramas por Segundo (FPS) promedio: {avg_fps:.2f} FPS")

print("\n" + "="*80)



                           RESUMEN FINAL DEL ANÁLISIS

Se detectaron un total de 113 heces en las imágenes analizadas.



,Categoría,Total de Detecciones,Imágenes y Detecciones
0,Sanas,93,"Imagen: 10.jpg (Detección #1), Imagen: 11.jpg (Detección #1), Imagen: 12.jpg (Detección #1), Imagen: 15.jpg (Detección #1), Imagen: 15.jpg (Detección #2), Imagen: 16.jpg (Detección #1), Imagen: 16.jpg (Detección #2), Imagen: 17.jpg (Detección #1), Imagen: 18.jpg (Detección #1), Imagen: 19.jpg (Detección #1), Imagen: 20.jpg (Detección #1), Imagen: 20.jpg (Detección #2), Imagen: 20.jpg (Detección #3), Imagen: 20.jpg (Detección #4), Imagen: 21.jpg (Detección #1), Imagen: 22.jpg (Detección #1), Imagen: 23.jpg (Detección #1), Imagen: 23.jpg (Detección #2), Imagen: 24.jpg (Detección #1), Imagen: 24.jpg (Detección #2), Imagen: 26.jpg (Detección #1), Imagen: 27.jpg (Detección #1), Imagen: 28.jpg (Detección #1), Imagen: 29.jpg (Detección #1), Imagen: 3.jpg (Detección #1), Imagen: 31.jpg (Detección #1), Imagen: 33.jpg (Detección #2), Imagen: 33.jpg (Detección #3), Imagen: 34.jpg (Detección #1), Imagen: 35.jpg (Detección #1), Imagen: 37.jpg (Detección #1), Imagen: 38.jpg (Detección #1), Imagen: 4.jpg (Detección #1), Imagen: 41.jpg (Detección #1), Imagen: 43.jpg (Detección #1), Imagen: 45.jpg (Detección #1), Imagen: 45.jpg (Detección #2), Imagen: 46.jpg (Detección #1), Imagen: 47.jpg (Detección #1), Imagen: 47.jpg (Detección #2), Imagen: 48.jpg (Detección #1), Imagen: 49.jpg (Detección #1), Imagen: 49.jpg (Detección #2), Imagen: 5.jpg (Detección #1), Imagen: 50.jpg (Detección #1), Imagen: 50.jpg (Detección #2), Imagen: 51.jpg (Detección #1), Imagen: 51.jpg (Detección #2), Imagen: 54.jpg (Detección #1), Imagen: 55.jpg (Detección #1), Imagen: 56.jpg (Detección #1), Imagen: 59.jpg (Detección #1), Imagen: 6.jpg (Detección #1), Imagen: 60.jpg (Detección #1), Imagen: 60.jpg (Detección #2), Imagen: 60.jpg (Detección #3), Imagen: 62.jpg (Detección #1), Imagen: 64.jpg (Detección #1), Imagen: 65.jpg (Detección #1), Imagen: 65.jpg (Detección #2), Imagen: 66.jpg (Detección #1), Imagen: 66.jpg (Detección #2), Imagen: 67.jpg (Detección #1), Imagen: 68.jpg (Detección #1), Imagen: 69.jpg (Detección #1), Imagen: 69.jpg (Detección #2), Imagen: 70.jpg (Detección #1), Imagen: 71.jpg (Detección #1), Imagen: 72.jpg (Detección #1), Imagen: 73.jpg (Detección #1), Imagen: 73.jpg (Detección #2), Imagen: 74.jpg (Detección #1), Imagen: 76.jpg (Detección #1), Imagen: 77.jpg (Detección #1), Imagen: 78.jpg (Detección #1), Imagen: 79.jpg (Detección #1), Imagen: 8.jpg (Detección #1), Imagen: 8.jpg (Detección #2), Imagen: 80.jpg (Detección #1), Imagen: 81.jpg (Detección #1), Imagen: 82.jpg (Detección #1), Imagen: 83.jpg (Detección #1), Imagen: 84.jpg (Detección #1), Imagen: 85.jpg (Detección #1), Imagen: 86.jpg (Detección #1), Imagen: 87.jpg (Detección #1), Imagen: 87.jpg (Detección #2), Imagen: 88.jpg (Detección #1), Imagen: 89.jpg (Detección #1), Imagen: 9.jpg (Detección #1), Imagen: 90.jpg (Detección #1), Imagen: 91.jpg (Detección #1), Imagen: 92.jpg (Detección #1)"
1,Coccidiosis,16,"Imagen: 1.jpg (Detección #1), Imagen: 14.jpg (Detección #1), Imagen: 2.jpg (Detección #1), Imagen: 32.jpg (Detección #1), Imagen: 33.jpg (Detección #1), Imagen: 39.jpg (Detección #1), Imagen: 40.jpg (Detección #1), Imagen: 44.jpg (Detección #1), Imagen: 44.jpg (Detección #2), Imagen: 44.jpg (Detección #3), Imagen: 52.jpg (Detección #1), Imagen: 52.jpg (Detección #2), Imagen: 63.jpg (Detección #1), Imagen: 71.jpg (Detección #2), Imagen: 81.jpg (Detección #2), Imagen: 82.jpg (Detección #2)"
2,Salmonella,4,"Imagen: 13.jpg (Detección #1), Imagen: 25.jpg (Detección #1), Imagen: 57.jpg (Detección #1), Imagen: 75.jpg (Detección #1)"
3,Newcastle Disease,0,




----------------------------------------
        MÉTRICAS DE VELOCIDAD PROMEDIO
----------------------------------------
  Imágenes analizadas: 93
  Tiempo promedio por imagen: 0.475 segundos
  Fotogramas por Segundo (FPS) promedio: 2.10 FPS

